<a href="https://colab.research.google.com/github/FranciscoRamirezArias/Pruebas_Software_Aseguramiento_Calidad/blob/main/ReservationSystem_v0_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!mkdir hotel_project


mkdir: cannot create directory ‘hotel_project’: File exists


Modelos

In [30]:
%%writefile hotel_project/models.py
"""
Domain models for Hotel Management System
"""

"""
Domain abstractions for Hotel Management System
"""

from dataclasses import dataclass, asdict
from typing import Dict


@dataclass
class Hotel:
    hotel_id: str
    name: str
    location: str
    total_rooms: int
    available_rooms: int

    def reserve_room(self) -> bool:
        """Decrease available rooms if possible."""
        if self.available_rooms > 0:
            self.available_rooms -= 1
            return True
        return False

    def display_info(self) -> str:
        return (
        f"\nHotel ID: {self.hotel_id}\n"
        f"Name: {self.name}\n"
        f"Location: {self.location}\n"
        f"Rooms: {self.available_rooms}/{self.total_rooms}\n"
    )

    def cancel_room(self) -> None:
        """Increase available rooms if below total."""
        if self.available_rooms < self.total_rooms:
            self.available_rooms += 1

    def update_info(self, name=None, location=None,
                    total_rooms=None) -> None:
        """Modify hotel information."""
        if name:
            self.name = name
        if location:
            self.location = location
        if total_rooms:
            self.total_rooms = total_rooms

    def to_dict(self) -> Dict:
        return asdict(self)


@dataclass
class Customer:
    customer_id: str
    name: str
    email: str
    phone: str

    def update_info(self, name=None, email=None,
                    phone=None) -> None:
        """Modify customer information."""
        if name:
            self.name = name
        if email:
            self.email = email
        if phone:
            self.phone = phone

    def to_dict(self) -> Dict:
        return asdict(self)


@dataclass
class Reservation:
    reservation_id: str
    customer_id: str
    hotel_id: str

    def to_dict(self) -> Dict:
        return asdict(self)


Overwriting hotel_project/models.py


Almacenamiento

In [21]:
%%writefile hotel_project/storage.py
"""
Persistent storage management (JSON files)
"""

import json
import os
from typing import List, Dict


def load_data(filename: str) -> List[Dict]:
    """Safely load JSON data."""
    if not os.path.exists(filename):
        return []

    try:
        with open(filename, "r", encoding="utf-8") as file:
            return json.load(file)
    except (json.JSONDecodeError, IOError) as error:
        print(f"Error reading {filename}: {error}")
        return []


def save_data(filename: str, data: List[Dict]) -> None:
    """Safely save JSON data."""
    try:
        with open(filename, "w", encoding="utf-8") as file:
            json.dump(data, file, indent=4)
    except IOError as error:
        print(f"Error writing {filename}: {error}")


Overwriting hotel_project/storage.py


Servicios

In [22]:
%%writefile hotel_project/services.py
"""
Service layer implementing persistent behaviors
"""

import uuid
from typing import Optional
from models import Hotel, Customer, Reservation
from storage import load_data, save_data


HOTELS_FILE = "hotels.json"
CUSTOMERS_FILE = "customers.json"
RESERVATIONS_FILE = "reservations.json"


class HotelService:

    @staticmethod
    def create_hotel(name: str, location: str,
                     total_rooms: int) -> Hotel:
        hotels = load_data(HOTELS_FILE)

        hotel = Hotel(
            hotel_id=str(uuid.uuid4()),
            name=name,
            location=location,
            total_rooms=total_rooms,
            available_rooms=total_rooms
        )

        hotels.append(hotel.to_dict())
        save_data(HOTELS_FILE, hotels)
        return hotel

    @staticmethod
    def delete_hotel(hotel_id: str) -> bool:
        hotels = load_data(HOTELS_FILE)
        updated = [h for h in hotels if h["hotel_id"] != hotel_id]

        if len(updated) == len(hotels):
            return False

        save_data(HOTELS_FILE, updated)
        return True

    @staticmethod
    def display_hotel(hotel_id: str) -> Optional[Hotel]:
        hotels = load_data(HOTELS_FILE)
        for hotel in hotels:
            if hotel["hotel_id"] == hotel_id:
                return Hotel(**hotel)
        return None

    @staticmethod
    def modify_hotel(hotel_id: str, **kwargs) -> bool:
        hotels = load_data(HOTELS_FILE)

        for hotel in hotels:
            if hotel["hotel_id"] == hotel_id:
                obj = Hotel(**hotel)
                obj.update_info(**kwargs)
                hotel.update(obj.to_dict())
                save_data(HOTELS_FILE, hotels)
                return True

        return False


class CustomerService:

    @staticmethod
    def create_customer(name: str, email: str,
                        phone: str) -> Customer:
        customers = load_data(CUSTOMERS_FILE)

        customer = Customer(
            customer_id=str(uuid.uuid4()),
            name=name,
            email=email,
            phone=phone
        )

        customers.append(customer.to_dict())
        save_data(CUSTOMERS_FILE, customers)
        return customer

    @staticmethod
    def delete_customer(customer_id: str) -> bool:
        customers = load_data(CUSTOMERS_FILE)
        updated = [c for c in customers
                   if c["customer_id"] != customer_id]

        if len(updated) == len(customers):
            return False

        save_data(CUSTOMERS_FILE, updated)
        return True

    @staticmethod
    def display_customer(customer_id: str) -> Optional[Customer]:
        customers = load_data(CUSTOMERS_FILE)
        for customer in customers:
            if customer["customer_id"] == customer_id:
                return Customer(**customer)
        return None

    @staticmethod
    def modify_customer(customer_id: str, **kwargs) -> bool:
        customers = load_data(CUSTOMERS_FILE)

        for customer in customers:
            if customer["customer_id"] == customer_id:
                obj = Customer(**customer)
                obj.update_info(**kwargs)
                customer.update(obj.to_dict())
                save_data(CUSTOMERS_FILE, customers)
                return True

        return False


class ReservationService:

    @staticmethod
    def create_reservation(customer_id: str,
                           hotel_id: str) -> Optional[Reservation]:

        hotels = load_data(HOTELS_FILE)
        reservations = load_data(RESERVATIONS_FILE)

        for hotel in hotels:
            if hotel["hotel_id"] == hotel_id:
                obj = Hotel(**hotel)

                if not obj.reserve_room():
                    return None

                hotel.update(obj.to_dict())
                save_data(HOTELS_FILE, hotels)

                reservation = Reservation(
                    reservation_id=str(uuid.uuid4()),
                    customer_id=customer_id,
                    hotel_id=hotel_id
                )

                reservations.append(reservation.to_dict())
                save_data(RESERVATIONS_FILE, reservations)
                return reservation

        return None

    @staticmethod
    def cancel_reservation(reservation_id: str) -> bool:
        reservations = load_data(RESERVATIONS_FILE)
        hotels = load_data(HOTELS_FILE)

        for reservation in reservations:
            if reservation["reservation_id"] == reservation_id:

                hotel_id = reservation["hotel_id"]

                reservations = [
                    r for r in reservations
                    if r["reservation_id"] != reservation_id
                ]

                for hotel in hotels:
                    if hotel["hotel_id"] == hotel_id:
                        obj = Hotel(**hotel)
                        obj.cancel_room()
                        hotel.update(obj.to_dict())

                save_data(RESERVATIONS_FILE, reservations)
                save_data(HOTELS_FILE, hotels)
                return True

        return False


Overwriting hotel_project/services.py


In [23]:
%%writefile hotel_project/main.py
"""
Main program for Hotel Management System
"""

from services import (
    HotelService,
    CustomerService,
    ReservationService
)


def main_menu():
    print("\n==== HOTEL MANAGEMENT SYSTEM ====")
    print("1. Hotel Management")
    print("2. Customer Management")
    print("3. Reservation Management")
    print("4. Exit")


# ---------------- HOTEL MENU ---------------- #

def hotel_menu():
    print("\n---- HOTEL MANAGEMENT ----")
    print("1. Create Hotel")
    print("2. Delete Hotel")
    print("3. Display Hotel Information")
    print("4. Modify Hotel")
    print("5. Back")


def handle_hotel():
    while True:
        hotel_menu()
        option = input("Select an option: ")

        try:
            if option == "1":
                name = input("Hotel name: ")
                location = input("Location: ")
                total_rooms = int(input("Total rooms: "))

                hotel = HotelService.create_hotel(
                    name,
                    location,
                    total_rooms
                )
                print(f"Hotel created with ID: {hotel.hotel_id}")

            elif option == "2":
                hotel_id = input("Hotel ID: ")
                result = HotelService.delete_hotel(hotel_id)
                print("Hotel deleted." if result else "Hotel not found.")

            elif option == "3":
                hotel_id = input("Hotel ID: ")
                hotel = HotelService.display_hotel(hotel_id)

                if hotel:
                    print(hotel.display_info())
                else:
                    print("Hotel not found.")

            elif option == "4":
                hotel_id = input("Hotel ID: ")
                name = input("New name (leave blank to skip): ")
                location = input("New location (leave blank to skip): ")
                total_rooms_input = input(
                    "New total rooms (leave blank to skip): "
                )

                kwargs = {}
                if name:
                    kwargs["name"] = name
                if location:
                    kwargs["location"] = location
                if total_rooms_input:
                    kwargs["total_rooms"] = int(total_rooms_input)

                result = HotelService.modify_hotel(
                    hotel_id,
                    **kwargs
                )
                print("Hotel updated." if result else "Hotel not found.")

            elif option == "5":
                break

            else:
                print("Invalid option.")

        except ValueError:
            print("Invalid numeric input.")
        except Exception as error:
            print(f"Unexpected error: {error}")


# ---------------- CUSTOMER MENU ---------------- #

def customer_menu():
    print("\n---- CUSTOMER MANAGEMENT ----")
    print("1. Create Customer")
    print("2. Delete Customer")
    print("3. Display Customer Information")
    print("4. Modify Customer")
    print("5. Back")


def handle_customer():
    while True:
        customer_menu()
        option = input("Select an option: ")

        try:
            if option == "1":
                name = input("Customer name: ")
                email = input("Email: ")
                phone = input("Phone: ")

                customer = CustomerService.create_customer(
                    name,
                    email,
                    phone
                )
                print(f"Customer created with ID: {customer.customer_id}")

            elif option == "2":
                customer_id = input("Customer ID: ")
                result = CustomerService.delete_customer(customer_id)
                print("Customer deleted."
                      if result else "Customer not found.")

            elif option == "3":
                customer_id = input("Customer ID: ")
                customer = CustomerService.display_customer(
                    customer_id
                )

                if customer:
                    print(customer)
                else:
                    print("Customer not found.")

            elif option == "4":
                customer_id = input("Customer ID: ")
                name = input("New name (leave blank to skip): ")
                email = input("New email (leave blank to skip): ")
                phone = input("New phone (leave blank to skip): ")

                kwargs = {}
                if name:
                    kwargs["name"] = name
                if email:
                    kwargs["email"] = email
                if phone:
                    kwargs["phone"] = phone

                result = CustomerService.modify_customer(
                    customer_id,
                    **kwargs
                )
                print("Customer updated."
                      if result else "Customer not found.")

            elif option == "5":
                break

            else:
                print("Invalid option.")

        except Exception as error:
            print(f"Unexpected error: {error}")


# ---------------- RESERVATION MENU ---------------- #

def reservation_menu():
    print("\n---- RESERVATION MANAGEMENT ----")
    print("1. Create Reservation")
    print("2. Cancel Reservation")
    print("3. Back")


def handle_reservation():
    while True:
        reservation_menu()
        option = input("Select an option: ")

        try:
            if option == "1":
                customer_id = input("Customer ID: ")
                hotel_id = input("Hotel ID: ")

                reservation = (
                    ReservationService.create_reservation(
                        customer_id,
                        hotel_id
                    )
                )

                if reservation:
                    print(
                        f"Reservation created with ID: "
                        f"{reservation.reservation_id}"
                    )
                else:
                    print("Reservation failed.")

            elif option == "2":
                reservation_id = input("Reservation ID: ")
                result = ReservationService.cancel_reservation(
                    reservation_id
                )
                print("Reservation cancelled."
                      if result else "Reservation not found.")

            elif option == "3":
                break

            else:
                print("Invalid option.")

        except Exception as error:
            print(f"Unexpected error: {error}")


# ---------------- MAIN LOOP ---------------- #

def main():
    while True:
        main_menu()
        option = input("Select an option: ")

        if option == "1":
            handle_hotel()
        elif option == "2":
            handle_customer()
        elif option == "3":
            handle_reservation()
        elif option == "4":
            print("Exiting system.")
            break
        else:
            print("Invalid option.")


if __name__ == "__main__":
    main()


Overwriting hotel_project/main.py


Ejecucion de Programa

In [32]:
!python hotel_project/main.py


==== HOTEL MANAGEMENT SYSTEM ====
1. Hotel Management
2. Customer Management
3. Reservation Management
4. Exit
Select an option: 1

---- HOTEL MANAGEMENT ----
1. Create Hotel
2. Delete Hotel
3. Display Hotel Information
4. Modify Hotel
5. Back
Select an option: 3
Hotel ID: 3

Hotel ID: 3
Name: Pinky
Location: USa
Rooms: 10/12


---- HOTEL MANAGEMENT ----
1. Create Hotel
2. Delete Hotel
3. Display Hotel Information
4. Modify Hotel
5. Back
Select an option: 5

==== HOTEL MANAGEMENT SYSTEM ====
1. Hotel Management
2. Customer Management
3. Reservation Management
4. Exit
Select an option: 4
Exiting system.
